In [4]:
### Load Documents
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredPowerPointLoader
import os

file_extension = ".pptx"
directory_path = "/home/datascience/demo/pptx"
documents = []

for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if filename.endswith(file_extension) and os.path.isfile(file_path):
        loader = UnstructuredPowerPointLoader(file_path)
        documents.extend(loader.load())

# The 'data' variable now holds a list of LangChain Document objects
# You can inspect the content and metadata of the documents:
#print(documents[0].page_content)
#print(documents[0].metadata)

## Split Text
# Initialize the RecursiveCharacterTextSplitter
# chunk_size: The maximum size of each chunk (in characters by default).
# chunk_overlap: The number of characters to overlap between consecutive chunks,
#                helping to maintain context.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,  # Use character length for chunk size
    is_separator_regex=False, # Treat separators literally
)

# Split the loaded documents
split_docs = text_splitter.split_documents(documents)

# Print the resulting chunks
print(f"Number of original documents: {len(documents)}")
print(f"Number of split chunks: {len(split_docs)}\n")

#for i, chunk in enumerate(split_docs):
#    print(f"Chunk {i+1}:\n{chunk.page_content}\n---")


{'source': '/home/datascience/demo/pptx/nacie_integration_team_meeting_20251009.pptx'}
Number of original documents: 3
Number of split chunks: 588



In [5]:
### Connect to OCI embeddings and generative AI
from langchain_community.embeddings import OCIGenAIEmbeddings
from langchain_community.chat_models import ChatOCIGenAI

oci_embeddings = OCIGenAIEmbeddings(
    model_id="cohere.embed-english-light-v3.0",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id="ocid1.compartment.oc1..aaaaaaaa52sp42nqmtwwzzvmp5mmldri26razhrbyw7cvixmims7p5crsg7a",
)

oci_chat = ChatOCIGenAI(
    model_id="cohere.command-a-03-2025",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id="ocid1.compartment.oc1..aaaaaaaa52sp42nqmtwwzzvmp5mmldri26razhrbyw7cvixmims7p5crsg7a",
    model_kwargs={"temperature": 0.7, "max_tokens": 500},
)

print("Connected to OCI embeddings and generative AI.")

#l = len(split_docs)
#embeddings = []
#for i in range(l // 16 + 1):
#    #print(f"Embedding from index {i*16} to {(i + 1) * 16}...")
#    subdocs = [item.page_content for item in split_docs[i * 16: (i + 1) * 16]]
#    embeddings.extend(oci_embeddings.embed_documents(subdocs))



Connected to OCI embeddings and generative AI.


In [6]:
### Build Vector Store (FAISS)
from langchain_community.vectorstores import FAISS

#texts = [item.page_content for item in split_docs]
#text_embedding_pairs = [(text, embed) for text, embed in zip(texts, embeddings)]
#vectorstore = FAISS.from_embeddings(text_embedding_pairs, oci_embeddings)
vectorstore = FAISS.from_documents(split_docs, oci_embeddings)

print("FAISS vector store built.")

FAISS vector store built.


In [7]:
### Build Chain (OCI chat-based Retrieval QA)
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains import RetrievalQA

retriever = vectorstore.as_retriever()

#rag_prompt_template = """Answer the question based only on the following context:
#{context}
#Question: {question}
#"""

#rag_prompt_template = """Try to the question using only the following context, but, if that fails, use your general knowledge.  You don't have to mention whether you did or did not use the context.
#{context}
#Question: {question}
#"""

rag_prompt_template = """Try to the question using only the following context, but, if that fails, use your general knowledge.
{context}
Question: {question}
"""

rag_prompt = PromptTemplate.from_template(rag_prompt_template)


#rag = RetrievalQA.from_chain_type(
#    llm=oci_chat,
#    retriever=retriever,
#    chain_type_kwargs={"prompt": rag_prompt,},
#)

rag = RetrievalQA.from_chain_type(
    llm=oci_chat,
    retriever=retriever,
)

print("RAG chain built.")

RAG chain built.


In [8]:
### Invoke the chain (unit test)
print(rag.invoke("What are the 4 pillars of performance?"))
print(rag.invoke("Why is Productivity important?"))
print(rag.invoke("Can I ignore any of the 4 pillars of performance?"))

{'query': 'What are the 4 pillars of performance?', 'result': 'The 4 pillars of performance, as mentioned in the provided context, are:\n\n1. **Growth**  \n2. **Productivity**  \n3. **Health**  \n4. **Innovation**  \n\nThese pillars form the foundation of the "Evolving 4-Pillar Approach" to performance criteria.'}
{'query': 'Why is Productivity important?', 'result': "Productivity is important because it measures an individual's contribution to revenue. This metric helps assess how effectively an individual is generating value for the organization. By tracking productivity, companies can identify areas for improvement, optimize resource allocation, and ensure that employees are aligned with organizational goals. It also provides a basis for evaluating performance and making informed decisions about growth and development."}
{'query': 'Can I ignore any of the 4 pillars of performance?', 'result': 'Based on the provided context, it seems that the 4-Pillar Approach (Growth, Productivity, 

In [9]:
### Use ADS to deploy the unit-tested chain
## Create the ADS deployment object
import tempfile
from ads.llm.deploy import ChainDeployment

artifact_dir = tempfile.mkdtemp()

ads_deployment = ChainDeployment(
    chain=rag,
    artifact_dir=artifact_dir,
    force_overwrite=True
)

ads_deployment.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Available     Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Not Available Local tested .predict from score.py                               
save()    Not Available Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [10]:
# Prepare the ADS deployment
ads_deployment.prepare(
    inference_conda_env="python_p312_any_x86_64_v1",
    inference_python_version="3.12",
)

# Summarize the checkpoitn ADS workflow status
ads_deployment.summary_status()

ERROR - Exception
Traceback (most recent call last):
  File "/home/datascience/conda/python_p312_any_x86_64_v1/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3670, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_23375/3364479652.py", line 2, in <module>
    ads_deployment.prepare(
  File "/home/datascience/conda/python_p312_any_x86_64_v1/lib/python3.12/site-packages/ads/llm/deploy.py", line 36, in prepare
    f.write(yaml.safe_dump(dump(self.chain)))
                           ^^^^^^^^^^^^^^^^
  File "/home/datascience/conda/python_p312_any_x86_64_v1/lib/python3.12/site-packages/ads/llm/serialize.py", line 209, in dump
    return __save(obj)
           ^^^^^^^^^^^
  File "/home/datascience/conda/python_p312_any_x86_64_v1/lib/python3.12/site-packages/ads/llm/serialize.py", line 184, in __save
    obj.save(temp_file.name)
  File "/home/datascience/conda/python_p312_any_x86_64_v1/lib/python3.12/site-packages/langchain_classic/